In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow import keras
import keras.backend as K
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, TensorBoard,LearningRateScheduler,EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras import Sequential,optimizers,models
from tensorflow.keras.layers import Dropout,Input,Conv2D, GlobalAvgPool2D, BatchNormalization, Dense,AveragePooling2D,MaxPool2D,ReLU,add
import warnings
warnings.filterwarnings("ignore")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
train_dir = '/home/kesci/work/data'
validation_dir = '/home/kesci/work/validata'
# data_dir = '/home/kesci/work/data'

In [3]:
batch_size = 32
width = 224
height = 224
input_shape = (height, width, 3)

In [4]:
train = ImageDataGenerator(
    rescale=1/255.0,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation = ImageDataGenerator(
      rescale=1./255)
      # rotation_range=40,
      # width_shift_range=0.2,
      # height_shift_range=0.2,
      # shear_range=0.2,
      # zoom_range=0.2,
      # horizontal_flip=True,
      # fill_mode='nearest'

In [5]:
train_ds = train.flow_from_directory(
        train_dir,
        target_size=(height, width),
        seed=3090,
        batch_size=batch_size,
        class_mode='categorical')

val_ds = validation.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        seed=3090,
        batch_size=batch_size,
        class_mode='categorical')
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         data_dir,
#         validation_split=0.2,
#         subset="training",
#         seed=3090,
#         image_size=(height, width),
#         batch_size=batch_size)
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         data_dir,
#         validation_split=0.2,
#         subset="validation",
#         seed=3090,
#         image_size=(height, width),
#         batch_size=batch_size)

Found 143667 images belonging to 3 classes.
Found 35917 images belonging to 3 classes.


In [6]:
NUM_TRAIN = 143667 
NUM_TEST = 35917

In [7]:
# n_epochs = 100
# # learning_rate = 1e-3
# n_cycles = n_epochs / 10
# lrate_max = 3e-4
# def scheduler(epoch):
#     epochs_per_cycle = n_epochs // n_cycles
#     cos_inner = (np.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
#     return lrate_max/2 * (np.cos(cos_inner) + 1)

# adam = optimizers.Adam(lr=lrate_max)
# change_lr = LearningRateScheduler(scheduler)

In [8]:
# class SnapshotEnsemble(tf.keras.callbacks.Callback):
#   def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
#     self.epochs = n_epochs
#     self.cycles = n_cycles
#     self.lr_max = lrate_max
#     self.lrates = list()

#   def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
#     epochs_per_cycle = n_epochs // n_cycles
#     cos_inner = (np.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
#     return lrate_max/2 * (np.cos(cos_inner) + 1)

#   def on_epoch_begin(self, epoch, logs={}):
#     lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
#     print(f'epoch {epoch+1}, lr {lr}')
#     K.set_value(self.model.optimizer.lr, lr)
#     self.lrates.append(lr)

#   def on_epoch_end(self, epoch, logs={}):
#     epochs_per_cycle = n_epochs // n_cycles
#     if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
#       filename = f"work/quick_add/snapshot_model_{int((epoch+1) / epochs_per_cycle)}.h5"
#       self.model.save(filename)
#       print(f'>saved snapshot {filename}, epoch {epoch}')

# n_epochs = 30
# n_cycles = n_epochs / 15
# ca = SnapshotEnsemble(n_epochs, n_cycles, 1e-3)

In [9]:
# covn_base = tf.keras.applications.EfficientNetB5(weights='imagenet',
#                                            input_shape=input_shape,
#                                            include_top=False,
#                                            pooling='avg')
# model = tf.keras.Sequential()
# model.add(covn_base)
# model.add(Dense(3,activation='softmax'))
model = keras.models.load_model('/home/kesci/work/eff_b5_other/model_weights03-0.79.h5')

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-5),
              metrics=['accuracy'])

In [10]:
# score = model.evaluate_generator(val_ds, verbose=1)

In [11]:
checkpoint = ModelCheckpoint('/home/kesci/work/eff_b5_best/model_weights{epoch:02d}-{val_accuracy:.2f}.h5', 
monitor='val_accuracy', verbose=10, save_best_only=True, mode='max')

callback_lists=[checkpoint]

# history = model.fit(
#   train_ds,
#   validation_data=val_ds,
#   epochs=n_epochs,
#   callbacks=callback_lists,
# )

history_tl = model.fit_generator(       
      train_ds,
      steps_per_epoch= NUM_TRAIN //batch_size,
      epochs=12,
      validation_data=val_ds,
      validation_steps= NUM_TEST //batch_size,
      verbose=1,
      use_multiprocessing=True,
      callbacks=callback_lists
      # workers=4
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/12
4489/4489 [==============================] - ETA: 0s - loss: 0.3975 - accuracy: 0.8329WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: val_accuracy improved from -inf to 0.78966, saving model to /home/kesci/work/eff_b5_best/model_weights01-0.79.h5
4489/4489 [==============================] - 2479s 552ms/step - loss: 0.3975 - accuracy: 0.8329 - val_loss: 0.5297 - val_accuracy: 0.7897
Epoch 2/12
4489/4489 [==============================] - ETA: 0s - loss: 0.3875 - accuracy: 0.8377WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: val_accuracy did not improve from 0.78966
4489/4489 [==============================] - 2340s 521ms/step - loss: 0.387

Process Keras_worker_ForkPoolWorker-9:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/utils/data_utils.py", line 679, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/opt/conda/lib/python3.8/site-packages/keras_preprocessing/image/iterator.py", line 65, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/opt/conda/lib/python3.8/site-packages/keras_preprocessing/image/iterator.py", line 238, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/opt/conda/lib/python3.8/site-packages/keras_p

KeyboardInterrupt: 

In [ ]:
# model.save('work/eff_b5_best/my_model.h5')

In [ ]:
# test_images /= 255.
# model.predict(test_images)